<a href="https://colab.research.google.com/github/fabiobento/dnn-course-2024-1/blob/main/00_course_folder/cert_prof_convnets/class_04/12%20-%20Atividade%20Avaliativa/C2W4_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

adaptado de [Certificado Profissional Desenvolvedor do TensorFlow](https://www.coursera.org/professional-certificates/tensorflow-in-practice) de [Laurence Moroney](https://laurencemoroney.com/)

# Classificação multiclasse

Nesta atividade, você terá a oportunidade de trabalhar em um problema de classificação multiclasse. Você usará o conjunto de dados [Sign Language MNIST](https://www.kaggle.com/datamunge/sign-language-mnist), que contém 28x28 imagens de mãos representando as 26 letras do alfabeto inglês. 

Você precisará pré-processar os dados para que possam ser inseridos em sua rede neural convolucional para classificar corretamente cada imagem como a letra que ela representa.


Vamos começar!

In [ ]:
import csv
import string
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img

Faça o download dos conjuntos de treinamento e teste (o conjunto de teste será, na verdade, usado como um conjunto de validação):

In [ ]:
!pip install gdown==5.1.0

In [ ]:
# Sign_mnist_train.csv
!gdown --id 1z0DkA9BytlLxO1C0BAWzknLyQmZAp0HR
# sign_mnist_test.csv
!gdown --id 1z1BIj4qmri59GWBG4ivMNFtpZ4AXIbzg

Defina algumas variáveis globais com o caminho para os dois arquivos que você acabou de baixar:

In [ ]:
TRAINING_FILE = './sign_mnist_train.csv'
VALIDATION_FILE = './sign_mnist_test.csv'

Diferentemente das tarefas anteriores, você não terá as imagens reais fornecidas; em vez disso, terá os dados serializados como arquivos `csv`.

Dê uma olhada na aparência dos dados no arquivo `csv`:

In [ ]:
with open(TRAINING_FILE) as training_file:
  line = training_file.readline()
  print(f"First line (header) looks like this:\n{line}")
  line = training_file.readline()
  print(f"Each subsequent line (data points) look like this:\n{line}")

Como você pode ver, cada arquivo inclui um cabeçalho (a primeira linha) e cada ponto de dados subsequente é representado como uma linha que contém 785 valores. 

O primeiro valor é o rótulo (a representação numérica de cada letra) e os outros 784 valores são o valor de cada pixel da imagem. 

Lembre-se de que as imagens originais têm uma resolução de 28x28, o que totaliza 784 pixels.

 ## Analisando o conjunto de dados
 
 Agora, complete a função `parse_data_from_input` abaixo.

 Essa função deve ser capaz de ler um arquivo passado como entrada e retornar 2 arrays numpy, um contendo os rótulos e outro contendo a representação 28x28 de cada imagem no arquivo. Esses arrays numéricos devem ser do tipo `float64`.

 Alguns aspectos a serem considerados:
 
- A primeira linha contém os cabeçalhos das colunas, portanto, você deve ignorá-la.

- Cada linha sucessiva contém 785 valores separados por vírgulas entre 0 e 255
  - O primeiro valor é o rótulo

  - Os demais são os valores de pixel para essa imagem
  
**Dica**:

Você tem duas opções para resolver essa função. 
  
   - 1. uma é usar `csv.reader` e criar um loop for que leia a partir dele; se você adotar essa abordagem, leve em consideração o seguinte:

        - O `csv.reader` retorna um iterável que retorna uma linha do arquivo csv em cada iteração.
    Seguindo essa convenção, row[0] tem o rótulo e row[1:] tem os 784 valores de pixel.

        - Para remodelar os arrays (passando de 784 para 28x28), você pode usar funções como [`np.array_split`](https://numpy.org/doc/stable/reference/generated/numpy.array_split.html) ou [`np.reshape`](https://numpy.org/doc/stable/reference/generated/numpy.reshape.html).

        - Para a conversão de tipo dos arrays numpy, use o método [`np.ndarray.astype`](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.astype.html).


   - 2) A outra opção é usar o `np.loadtxt`. Você pode encontrar a documentação [aqui](https://numpy.org/doc/stable/reference/generated/numpy.loadtxt.html).
   
   
Independentemente do método escolhido, sua função deve terminar a execução em menos de 1 minuto. Se perceber que sua função está demorando muito para ser executada, tente alterar sua implementação. 

In [ ]:
def parse_data_from_input(filename):
  """
  Analisa as imagens e os rótulos de um arquivo CSV
  
  Args:
    filename (string): caminho para o arquivo CSV
    
  Retorna:
    images, labels: tupla de matrizes numpy contendo as imagens e os rótulos
  """
  with open(filename) as file:
    ### COMECE O CÓDIGO AQUI

    # Use csv.reader, passando o delimitador apropriado
    # Lembre-se de que o csv.reader pode ser iterado e retorna uma linha em cada iteração
    csv_reader = csv.reader(file, delimiter=None)
    
    labels = None
    images = None

    
    ### TERMINE O CÓDIGO AQUI

    return images, labels

In [ ]:
# Teste sua função
training_images, training_labels = parse_data_from_input(TRAINING_FILE)
validation_images, validation_labels = parse_data_from_input(VALIDATION_FILE)

print(f"As imagens de treino tem o formato: {training_images.shape} e tipo: {training_images.dtype}")
print(f"Os rótulos de treino tem o formato: {training_labels.shape} e tipo: {training_labels.dtype}")
print(f"As imagens de validação tem o formato: {validation_images.shape} e tipo: {validation_images.dtype}")
print(f"Os rótulos de validação tem o formato: {validation_labels.shape} e tipo: {validation_labels.dtype}")

**Saída Esperada:**
```
As imagens de treino tem o formato: (27455, 28, 28) e tipo: float64
Os rótulos de treino tem o formato: (27455,) e tipo: float64
As imagens de validação tem o formato: (7172, 28, 28) e tipo: float64
Os rótulos de validação tem o formato: (7172,) e tipo: float64
```

## Visualizando as matrizes numpy

Agora que você converteu os dados csv iniciais em um formato compatível com as tarefas de visão computacional, reserve um momento para ver como são as imagens do conjunto de dados:

In [ ]:
# Plote uma amostra de 10 imagens do conjunto de treinamento
def plot_categories(training_images, training_labels):
  fig, axes = plt.subplots(1, 10, figsize=(16, 15))
  axes = axes.flatten()
  letters = list(string.ascii_lowercase)

  for k in range(10):
    img = training_images[k]
    img = np.expand_dims(img, axis=-1)
    img = array_to_img(img)
    ax = axes[k]
    ax.imshow(img, cmap="Greys_r")
    ax.set_title(f"{letters[int(training_labels[k])]}")
    ax.set_axis_off()

  plt.tight_layout()
  plt.show()

plot_categories(training_images, training_labels)

## Criação dos geradores para a CNN

Agora que você organizou com êxito os dados de uma forma que pode ser facilmente alimentada pelo `ImageDataGenerator` do Keras, é hora de codificar os geradores que produzirão lotes de imagens, tanto para treinamento quanto para validação. Para isso, complete a função `train_val_generators` abaixo.

Algumas observações importantes:

- As imagens desse conjunto de dados vêm com a mesma resolução, portanto, você não precisa definir um `target_size` personalizado nesse caso. Na verdade, você nem mesmo pode fazer isso porque, desta vez, não usará o método `flow_from_directory` (como nas atribuições anteriores). Em vez disso, você usará o método [`flow`](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator#flow).
- Você precisa adicionar a dimensão "color" aos arrays numpy que codificam as imagens. Essas imagens são em preto e branco, portanto, essa nova dimensão deve ter um tamanho de 1 (em vez de 3, que é usado quando se lida com imagens coloridas). Para isso, dê uma olhada na função [`np.expand_dims`](https://numpy.org/doc/stable/reference/generated/numpy.expand_dims.html).

In [ ]:
def train_val_generators(training_images, training_labels, validation_images, validation_labels):
  """
  Cria os geradores de dados de treinamento e validação
  
  Args:
    training_images (array): imagens analisadas do arquivo CSV de treinamento
    training_labels (matriz): rótulos analisados do arquivo CSV de treinamento
    validation_images (matriz): imagens analisadas do arquivo CSV de teste
    validation_labels (matriz): rótulos analisados do arquivo CSV de teste
    
  Retorna:
    train_generator, validation_generator - tupla contendo os geradores
  """
  ### COMECE O CÓDIGO AQUI

  # Nesta seção, você terá de adicionar outra dimensão aos dados
  # Então, por exemplo, se sua matriz for (10000, 28, 28)
  # Você precisará torná-lo (10000, 28, 28, 1)
  # Dica: np.expand_dims

  training_images = None
  validation_images = None

  # Instanciar a classe ImageDataGenerator 
  # Não se esqueça de normalizar os valores de pixel 
  # e definir argumentos para aumentar as imagens (se desejado)
  train_datagen = None


  # Passe os argumentos apropriados para o método de fluxo
  train_generator = train_datagen.flow(x=None,
                                       y=None,
                                       batch_size=32) 

  
  # Instanciar a classe ImageDataGenerator (não se esqueça de definir o argumento rescale)
  # Lembre-se de que os dados de validação não devem ser aumentados
  validation_datagen = None

  # Passe os argumentos apropriados para o método de fluxo
  validation_generator = validation_datagen.flow(x=None,
                                                 y=None,
                                                 batch_size=32) 

  ### TERMINE SEU CÓDIGO AQUI

  return train_generator, validation_generator

In [ ]:
# Teste seus geradores
train_generator, validation_generator = train_val_generators(training_images, training_labels, validation_images, validation_labels)

print(f"As imagens de treino do gerador têm forma: {train_generator.x.shape}")
print(f"Os rótulos de treino do gerador têm forma: {train_generator.y.shape}")
print(f"As imagens de validação do gerador têm forma: {validation_generator.x.shape}")
print(f"Os rótulos de validação do gerador têm forma: {validation_generator.y.shape}")

**Saída Esperada:**
```
As imagens de treino do gerador têm forma: (27455, 28, 28, 1)
Os rótulos de treino do gerador têm forma: (27455,)
Os rótulos de validação do gerador têm forma: (7172, 28, 28, 1)
Os rótulos de validação do gerador têm forma: (7172,)
```

## Codificação da CNN

Uma última etapa antes do treinamento é definir a arquitetura do modelo.

Complete a função `create_model` abaixo. Essa função deve retornar um modelo do Keras que use a API `Sequential` ou `Functional`.

A última camada de seu modelo deve ter um número de unidades igual ao número de letras do alfabeto inglês. Ela também deve usar uma função de ativação que produzirá as probabilidades por letra.

*Observação: A [documentation](https://www.kaggle.com/datamunge/sign-language-mnist) do conjunto de dados menciona que, na verdade, não há casos para a última letra, Z, e isso permitirá que você reduza em um o número recomendado de unidades de saída acima. Se ainda não estiver convencido, pode ignorar esse fato com segurança por enquanto e estudá-lo mais tarde. Você será aprovado no trabalho mesmo sem essa pequena otimização.*

Além de definir a arquitetura do modelo, você também deve compilá-lo, portanto, certifique-se de usar uma função `loss` que seja adequada para classificação multiclasse.

**Observe que você não deve usar mais do que 2 camadas Conv2D e 2 camadas MaxPooling2D para obter o desempenho desejado.**

In [ ]:
def create_model():

  ### INICIE O CÓDIGO AQUI       

  # Definir o modelo
  # Não use mais do que 2 Conv2D e 2 MaxPooling2D

  model = None
  

  model.compile(optimizer = None,
                loss = None,
                metrics=[None])

  ### TERMINE SEU CÓDIGO AQUI
  
  return model

In [ ]:
# Salve seu modelo
model = create_model()

# Treine seu modelo
history = model.fit(train_generator,
                    epochs=15,
                    validation_data=validation_generator)

Agora, dê uma olhada no seu histórico de treinamento:

In [ ]:
# Trace o gráfico de precisão e perda no treinamento e na validação
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Acurácia de Treino')
plt.plot(epochs, val_acc, 'b', label='Acurácia de Validação')
plt.title('Acurácia de Treino e Validação')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'r', label='Perda de Treino')
plt.plot(epochs, val_loss, 'b', label='Perda de Validação')
plt.title('Perda de treino e validação')
plt.legend()

plt.show()

Você não será avaliado com base na acurácia do seu modelo, mas tente torná-la a mais alta possível tanto para treinamento quanto para validação, como um exercício opcional, **após enviar seu notebook para avaliação**.

Uma referência razoável é atingir mais de 99% de precisão para treinamento e mais de 95% de precisão para validação em 15 épocas. Tente ajustar a arquitetura do seu modelo ou as técnicas de aumento para ver se você consegue atingir esses níveis de acurácia.

**Parabéns por terminar concluir a tarefa!**

Você implementou com sucesso uma rede neural convolucional que é capaz de executar tarefas de classificação multiclasse! Bom trabalho!